In [1]:
!pip -q install selenium webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 25.4 MB/s eta 0:00:00


In [1]:
# Colab: Google Chrome を導入 → Selenium を最新化
!apt-get update -y
!apt-get install -y wget gnupg
!wget -q https://dl.google.com/linux/linux_signing_key.pub -O- | apt-key add -
!bash -lc 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" > /etc/apt/sources.list.d/google-chrome.list'
!apt-get update -y
!apt-get install -y google-chrome-stable
!python -m pip -q install --upgrade selenium

!google-chrome --version


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://cli.github.com/packages stable InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not s

In [6]:
# ================================================
# SAMURAI TERAKOYA 自己紹介を自動更新（完全版・Colab/ローカル両対応・1セル）
# ================================================
import os, sys, shutil, subprocess, shlex, time
from pathlib import Path
from getpass import getpass

# ----- 実行環境の判定 -----
IN_COLAB = ("google.colab" in sys.modules) or ("COLAB_GPU" in os.environ)

def _run(cmd: str):
    p = subprocess.run(shlex.split(cmd), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    print(f"$ {cmd}\n{p.stdout}")
    return p.returncode == 0

# ----- Colab セットアップ（必要なときだけ動く） -----
if IN_COLAB:
    _run("apt-get update -y")
    _run("apt-get install -y wget gnupg")
    _run("bash -lc 'echo \"deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main\" > /etc/apt/sources.list.d/google-chrome.list'")
    _run("wget -qO- https://dl.google.com/linux/linux_signing_key.pub | apt-key add -")
    _run("apt-get update -y")
    _run("apt-get install -y google-chrome-stable")
    _run("python -m pip -q install --upgrade pip selenium")
    _run("google-chrome --version")

# ----- 定数 -----
ENTRY_URL   = "https://terakoya.sejuku.net/"
PROFILE_URL = "https://terakoya.sejuku.net/account/profile"
INTRO_TEXT  = "プログラミング学習中です！今はスクレイピングに挑戦しています！"

HEADLESS    = True      # 画面を見たいとき False
WINDOW_SIZE = "1280,900"
WAIT_S      = 15

SS_OK   = Path("after_profile_saved.png")
SS_ERR  = Path("error_timeout.png")
HTMLDBG = Path("page_debug.html")

# ----- Selenium import -----
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# ----- Driver 起動 -----
def _paths():
    return (
        shutil.which("google-chrome") or "/usr/bin/google-chrome",
        shutil.which("chromium-browser") or shutil.which("chromium"),
        shutil.which("chromedriver"),
    )

def make_driver(headless: bool = HEADLESS):
    """Selenium Manager で Chrome を優先。ダメなら Chromium + chromedriver にフォールバック。"""
    chrome_bin, chromium_bin, chromedriver_bin = _paths()

    def base_options():
        opts = Options()
        opts.add_argument("--window-size=" + WINDOW_SIZE)
        if headless:
            opts.add_argument("--headless=new")
        opts.add_argument("--disable-dev-shm-usage")
        opts.add_argument("--disable-gpu")
        opts.add_argument("--disable-extensions")
        opts.add_argument("--disable-notifications")
        opts.add_argument("--no-first-run")
        opts.add_argument("--no-default-browser-check")
        if IN_COLAB:
            opts.add_argument("--no-sandbox")
        # 軽量化：画像をロードしない
        opts.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})
        # ページロード完了を軽くする
        opts.page_load_strategy = "eager"
        return opts

    # 1) Google Chrome + Selenium Manager
    try:
        opts = base_options()
        if os.path.exists("/usr/bin/google-chrome"):
            opts.binary_location = "/usr/bin/google-chrome"
        driver = webdriver.Chrome(options=opts)  # Service を渡さない = Selenium Managerが良い感じに選定
        driver.set_page_load_timeout(30)
        print("✅ Driver: Google Chrome (Selenium Manager)")
        return driver
    except Exception as e:
        print("[Info] Chrome起動失敗。Chromiumで再試行:", type(e).__name__, e)

    # 2) Chromium + chromedriver
    if chromium_bin and chromedriver_bin:
        opts = base_options()
        opts.binary_location = chromium_bin
        opts.add_argument("--remote-debugging-port=9222")
        service = Service(chromedriver_bin)
        driver = webdriver.Chrome(service=service, options=opts)
        driver.set_page_load_timeout(30)
        print(f"✅ Driver: Chromium ({chromium_bin}) + chromedriver ({chromedriver_bin})")
        return driver

    raise RuntimeError("Chrome/Chromium の起動に失敗しました。ランタイム再起動後にもう一度実行してください。")

# ----- ユーティリティ -----
def _click_if_present(driver, xpath, timeout=8):
    """見つかったら確実にクリック（画面外→スクロール、透明でもJSクリック）"""
    try:
        el = WebDriverWait(driver, timeout).until(EC.presence_of_element_located((By.XPATH, xpath)))
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
        try:
            WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, xpath)))
            el.click()
        except Exception:
            driver.execute_script("arguments[0].click();", el)
        time.sleep(0.3)
        return True
    except Exception:
        return False

def open_entry(driver):
    driver.get(ENTRY_URL)
    WebDriverWait(driver, WAIT_S).until(
        EC.presence_of_element_located((By.XPATH, "//*[normalize-space(.)='ログイン' or normalize-space(.)='ログインする']"))
    )
    print("✅ open:", ENTRY_URL)

def click_or_open_login(driver):
    """a[href]直アクセス→ダメならヘッダー『ログイン』（モーダル dQOsDP 対応）"""
    wait = WebDriverWait(driver, WAIT_S)

    # a[href] から最短で飛べるなら飛ぶ
    try:
        anchors = driver.find_elements(By.XPATH,
            "//a[contains(normalize-space(.),'ログイン')]"
            "|//a[contains(@aria-label,'ログイン') or contains(@title,'ログイン')]")
        for a in anchors:
            href = (a.get_attribute("href") or "").strip()
            if href and href != "#" and "javascript:" not in href:
                driver.get(href)
                try:
                    WebDriverWait(driver, 6).until(EC.presence_of_element_located(
                        (By.XPATH, "//input[@type='email' or contains(@name,'mail') or contains(@id,'mail')]")
                    ))
                    print("✅ login直URLに到達:", href)
                    return
                except Exception:
                    pass
    except Exception:
        pass

    # テキスト一致でクリック（あなたの環境で安定）
    driver.execute_script("window.scrollTo(0,0);")
    try:
        el = wait.until(EC.element_to_be_clickable((By.XPATH,
            "//*[self::a or self::div or self::button][normalize-space(.)='ログイン' or normalize-space(.)='ログインする']")))
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
        el.click()
        print("✅ テキストでログインクリック（モーダル想定）")
    except Exception:
        pass

    # モーダル（dQOsDP クラス）を待つ
    try:
        WebDriverWait(driver, 12).until(EC.presence_of_element_located(
            (By.XPATH, "//div[contains(@class,'dQOsDP')]//input[@type='email' or contains(@name,'mail') or contains(@id,'mail')]")
        ))
        print("✅ 同一ページ内にログインフォームを検知（dQOsDP）")
        return
    except Exception:
        pass

    # iframe保険
    for idx, f in enumerate(driver.find_elements(By.TAG_NAME, "iframe")):
        try:
            driver.switch_to.default_content()
            driver.switch_to.frame(f)
            WebDriverWait(driver, 6).until(
                EC.presence_of_element_located((By.XPATH, "//input[@type='email' or contains(@name,'mail') or contains(@id,'mail')]"))
            )
            print(f"✅ iframe[{idx}] 内でログインフォームを検知")
            return
        except Exception:
            continue
    driver.switch_to.default_content()
    driver.save_screenshot(str(SS_ERR))
    HTMLDBG.write_text(driver.page_source, encoding="utf-8")
    raise TimeoutException("ログイン画面に到達できませんでした。")

def find_login_fields(driver):
    """メール/パス/送信ボタン（送信はNone可）"""
    wait = WebDriverWait(driver, WAIT_S)
    def _resolve():
        email_el = wait.until(EC.presence_of_element_located((By.XPATH,
            "//input[@type='email' or contains(@name,'mail') or contains(@id,'mail') or @autocomplete='username']")))
        pass_el  = wait.until(EC.presence_of_element_located((By.XPATH,
            "//input[@type='password' or contains(@name,'pass') or contains(@id,'pass') or @autocomplete='current-password']")))
        try:
            submit_el = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH,
                "//button[@type='submit' or contains(normalize-space(.),'ログイン') or contains(normalize-space(.),'サインイン')] | //input[@type='submit']")))
        except Exception:
            submit_el = None
        return email_el, pass_el, submit_el
    try:
        return _resolve()
    except Exception:
        pass
    for f in driver.find_elements(By.TAG_NAME, "iframe"):
        try:
            driver.switch_to.default_content(); driver.switch_to.frame(f)
            return _resolve()
        except Exception:
            continue
    driver.switch_to.default_content()
    raise TimeoutException("ログインフォームが見つかりません。")

def do_login(driver, email_value, password_value):
    open_entry(driver)
    click_or_open_login(driver)
    email_el, pass_el, submit_el = find_login_fields(driver)
    email_el.clear(); email_el.send_keys(email_value)
    pass_el.clear();  pass_el.send_keys(password_value)
    if submit_el:
        try: submit_el.click()
        except Exception: driver.execute_script("arguments[0].click();", submit_el)
    else:
        pass_el.send_keys(Keys.ENTER)
    # 成功らしき変化
    try:
        WebDriverWait(driver, WAIT_S).until(EC.any_of(
            EC.visibility_of_element_located((By.XPATH, "//*[contains(.,'ログアウト')]")),
            EC.url_changes(driver.current_url),
            EC.url_contains("/dashboard"), EC.url_contains("/home"),
        ))
    except Exception:
        pass

# ----- プロフィール自己紹介 更新（あなたのDOMに最適化）-----
def update_profile_intro(driver, text, timeout=WAIT_S):
    w = WebDriverWait(driver, timeout)
    driver.get(PROFILE_URL)

    # 「アカウント設定」タブ（/account/profile）に確実に遷移
    _click_if_present(driver, "//a[@href='/account/profile']", timeout=4) \
        or _click_if_present(driver, "//*[self::a or self::li or self::button][normalize-space(.)='アカウント設定']", timeout=4)

    # 「編集」ボタン
    _click_if_present(driver, "//button[normalize-space(.)='編集']", timeout=8)

    # 自己紹介の <textarea> を取得（あなたの検証で確認できたクラスを最優先）
    textarea = None
    candidates = [
        (By.CSS_SELECTOR, "textarea.sc-glwiet.kKaa-Dp"),  # スクショのクラス
        (By.XPATH, "//label[contains(.,'自己紹介')]/following::textarea[1]"),
        (By.XPATH, "//textarea[contains(@name,'intro') or contains(@name,'bio') or contains(@id,'intro') or contains(@id,'bio') or contains(@placeholder,'自己紹介')]"),
        (By.XPATH, "(//textarea)[1]"),
    ]

    last_err = None
    for by, sel in candidates:
        try:
            textarea = WebDriverWait(driver, 12).until(EC.visibility_of_element_located((by, sel)))
            break
        except Exception as e:
            last_err = e
            continue

    # 見切れている場合の再試行
    if textarea is None:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight * 0.5);")
        time.sleep(0.3)
        for by, sel in candidates:
            try:
                textarea = WebDriverWait(driver, 6).until(EC.visibility_of_element_located((by, sel)))
                break
            except Exception:
                continue

    if textarea is None:
        driver.save_screenshot(str(SS_ERR))
        HTMLDBG.write_text(driver.page_source, encoding="utf-8")
        raise TimeoutException("自己紹介テキストエリアが見つかりません。")

    # 入力（完全置換）
    try: textarea.clear()
    except Exception: pass
    textarea.click()
    textarea.send_keys(Keys.CONTROL, "a")
    textarea.send_keys(Keys.DELETE)
    textarea.send_keys(text)

    # 「更新する」ボタンで保存
    saved = False
    for xp in [
        "//button[normalize-space(.)='更新する']",
        "//button[contains(normalize-space(.),'保存') or contains(normalize-space(.),'変更を保存')]",
        "//input[@type='submit']",
    ]:
        if _click_if_present(driver, xp, timeout=8):
            saved = True
            break
    if not saved:
        textarea.send_keys(Keys.CONTROL, Keys.ENTER)

    # 成功らしき変化を待つ（編集ボタン復帰/完了文言/URL）
    try:
        WebDriverWait(driver, 10).until(EC.any_of(
            EC.visibility_of_element_located((By.XPATH, "//button[normalize-space(.)='編集']")),
            EC.visibility_of_element_located((By.XPATH, "//*[contains(.,'保存') and contains(.,'完了')]")),
            EC.visibility_of_element_located((By.XPATH, "//*[contains(.,'更新') and contains(.,'完了')]")),
            EC.url_contains("/account"),
        ))
    except Exception:
        pass

# ----- 実行 -----
email = input("メールアドレス: ")
try:
    password = getpass("パスワード: ")
except Exception:
    password = input("パスワード（表示されます）: ")

driver = make_driver(headless=HEADLESS)
try:
    do_login(driver, email, password)
    update_profile_intro(driver, INTRO_TEXT)
    time.sleep(1.0)
    driver.save_screenshot(str(SS_OK))
    print(f"✅ 自己紹介を更新しました。スクリーンショット: {SS_OK}")
except Exception as e:
    print("[Error] 想定外のエラー:", type(e).__name__, e)
    try:
        driver.save_screenshot(str(SS_ERR))
        HTMLDBG.write_text(driver.page_source, encoding="utf-8")
        print(f"🧪 デバッグ保存: {SS_ERR}, {HTMLDBG}")
    except Exception:
        pass
finally:
    try: driver.quit()
    except Exception: pass


$ apt-get update -y
Hit:1 http://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://cli.github.com/packages stable InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelea